#upload the data

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mernamansour95","key":"07a87b915766c812c294e39792463ac9"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:

!kaggle datasets download bittlingmayer/amazonreviews

Dataset URL: https://www.kaggle.com/datasets/bittlingmayer/amazonreviews
License(s): unknown
100% 491M/493M [00:23<00:00, 24.0MB/s]
100% 493M/493M [00:23<00:00, 21.7MB/s]


In [4]:
! unzip amazonreviews.zip


Archive:  amazonreviews.zip
  inflating: test.ft.txt.bz2         
  inflating: train.ft.txt.bz2        


In [5]:
! Importing necessary libraries
import spacy
import gensim
import numpy as np
import tensorflow as tf

from gensim.models import Word2Vec
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
import pandas as pd
import re
import bz2
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

/bin/bash: line 1: Importing: command not found


#read data

In [6]:
test_path = r"/content/test.ft.txt.bz2"
train_path = r"/content/train.ft.txt.bz2"

In [7]:
def load_data(file_path):
    data = []
    with bz2.open(file_path, 'rt', encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            line = line.strip()  # Remove leading/trailing spaces or newlines
            data.append(line)
    return pd.DataFrame(data, columns=['text'])

In [8]:
train_data = load_data(train_path)
test_data = load_data(test_path)

In [9]:
train_data.head()

,text
0,__label__2 Stuning even for the non-gamer: Thi...
1,__label__2 The best soundtrack ever to anythin...
2,__label__2 Amazing!: This soundtrack is my fav...
3,__label__2 Excellent Soundtrack: I truly like ...
4,"__label__2 Remember, Pull Your Jaw Off The Flo..."


In [10]:
test_data.head()

,text
0,__label__2 Great CD: My lovely Pat has one of ...
1,__label__2 One of the best game music soundtra...
2,__label__1 Batteries died within a year ...: I...
3,"__label__2 works fine, but Maha Energy is bett..."
4,__label__2 Great for the non-audiophile: Revie...


In [11]:
print("Train Data Shape:", train_data.shape)  # Print the shape of the DataFrame

Train Data Shape: (3600000, 1)


#data preprocessing


In [12]:
def preprocess_data(data):
    # Separate labels and review text
    data.columns = ['text']

    # Drop rows where text is NaN or not a string
    data = data.dropna(subset=['text'])
    data['text'] = data['text'].astype(str)  # Ensure all text is treated as a string

    # Extract labels (0 for __label__1, 1 for __label__2)
    data['label'] = data['text'].apply(lambda x: 0 if '__label__1' in x else 1)

    # Remove the label from the text
    data['text'] = data['text'].apply(lambda x: re.sub(r'__label__\d ', '', x))

    # Remove the title (text before ": ")
    data['text'] = data['text'].apply(lambda x: re.sub(r'^.*?: ', '', x))

    return data[['text', 'label']]


In [13]:
train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

In [14]:

print(train_data.columns)
print(test_data.columns)

Index(['text', 'label'], dtype='object')
Index(['text', 'label'], dtype='object')


In [14]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['text'])



In [15]:
# Convert text to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['text'])
test_sequences = tokenizer.texts_to_sequences(test_data['text'])

In [16]:
max_seq_length = 100
X_train = pad_sequences(train_sequences, maxlen=max_seq_length)
X_test = pad_sequences(test_sequences, maxlen=max_seq_length)

In [17]:

y_train = train_data['label'].values
y_test = test_data['label'].values

In [18]:
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

In [1]:
import numpy as np
# Load preprocessed data
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

#build the model

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = 5000
embedding_dim = 32
max_length = 100

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))  # Embedding layer
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))  # LSTM layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification



/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [3]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])




In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=32, validation_split=0.2)



Epoch 1/3
38898/90000 ━━━━━━━━━━━━━━━━━━━━ 1:44:07 122ms/step - accuracy: 0.8655 - loss: 0.3124

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Confusion matrix and classification report
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

#save the model

In [ ]:
model.save('text_model_1.keras')